In [1]:
%pip install avro
# import required libraries
from kafka import KafkaConsumer, KafkaProducer
import avro.schema
import avro.io
import io
import hashlib, json
import time
import requests

Note: you may need to restart the kernel to use updated packages.


In [2]:
def serialize(schema, obj):
    bytes_writer = io.BytesIO()
    encoder = avro.io.BinaryEncoder(bytes_writer)
    writer = avro.io.DatumWriter(schema)
    writer.write(obj, encoder)
    return bytes_writer.getvalue()

In [3]:
def deserialize(schema, raw_bytes):
    bytes_reader = io.BytesIO(raw_bytes)
    decoder = avro.io.BinaryDecoder(bytes_reader)
    reader = avro.io.DatumReader(schema)
    return reader.read(decoder)

In [4]:
schema_file = 'transaction.avsc'
txschema = avro.schema.parse(open(schema_file).read())
schema_file = 'submit.avsc'
submitschema = avro.schema.parse(open(schema_file).read())
schema_file = 'result.avsc'
resultschema = avro.schema.parse(open(schema_file).read())

In [5]:
# Connect to kafka broker running in your local host (docker). Change this to your kafka broker if needed
kafka_broker = '35.240.149.229:9092'

In [6]:
producer = KafkaProducer(bootstrap_servers=[kafka_broker])

In [7]:
txconsumer = KafkaConsumer(
    'transaction',
     bootstrap_servers=[kafka_broker],
     enable_auto_commit=True,
     value_deserializer=lambda x: deserialize(txschema, x))
resultconsumer = KafkaConsumer(
    'result',
     bootstrap_servers=[kafka_broker],
     enable_auto_commit=True,
     value_deserializer=lambda x: deserialize(resultschema, x))

In [8]:
def gen_signature(txid, payer, payee, amount, token):
    o = {'txid': txid, 'payer': payer, 'payee': payee, 'amount': amount, 'token': token}
    return hashlib.md5(json.dumps(o, sort_keys=True).encode('utf-8')).hexdigest()

In [9]:
class TransactionVerifier:
    def __init__(self):
        self.token = "42cdee0e79c04bd1ffd5f4c4efe59a89"
        self.vid = "V411709"	
        self.submit_schema = submitschema
        self.result_schema = resultschema
        self.tx_schema = txschema
        self.gg_formurl = ""

    def read_transaction(self, transaction):
        # Extract the transaction information
        txid = transaction['txid']
        payer = transaction['payer']
        payee = transaction['payee']
        amount = transaction['amount']

        # Generate the signature using gen_signature function 
        signature = gen_signature(txid, payer, payee, amount, self.token)

        submit_data = {'vid': self.vid , 'txid': txid, 'signature': signature}
        
        data = serialize(self.submit_schema, submit_data)
        print(submit_data)
        print(data)
        producer.send('submit', data)
    
    def receive_confirmation(self, result):
        # Check if the result code is 200 and the vid and txid match
        if result['code'] == 200 and result['vid'] == self.vid and result['txid'] == txid:
            # Send the confirmation to Google form
            requests.post(self.gg_formurl, data={'entry.your_field_id': 'Confirmation'})


In [10]:
trxVerifier = TransactionVerifier()

In [11]:
print('Running Trx Consumer... with AVRO')
for message in txconsumer:
    txid, payer, payee, amount= message.value.values()
    print(txid, payer, payee, amount)
    trxVerifier.read_transaction(message.value)

Running Trx Consumer... with AVRO
TX04028 A22647 A30746 871
{'vid': 'V411709', 'txid': 'TX04028', 'signature': '79aed7766caac9bd7b91532089848fb5'}
b'\x0eV411709\x0eTX04028@79aed7766caac9bd7b91532089848fb5'
TX06939 A59995 A25028 1789
{'vid': 'V411709', 'txid': 'TX06939', 'signature': '2860d9f9acbc7cd5447aa5b9e5b3b172'}
b'\x0eV411709\x0eTX06939@2860d9f9acbc7cd5447aa5b9e5b3b172'
TX04571 A41732 A65292 118
{'vid': 'V411709', 'txid': 'TX04571', 'signature': '7d749409129f0e49ce0cb78d6a629c3c'}
b'\x0eV411709\x0eTX04571@7d749409129f0e49ce0cb78d6a629c3c'
TX06627 A90417 A76574 1096
{'vid': 'V411709', 'txid': 'TX06627', 'signature': '3e66066dcdcd9575569b7ce18fe2e11a'}
b'\x0eV411709\x0eTX06627@3e66066dcdcd9575569b7ce18fe2e11a'
TX07763 A23071 A72908 1827
{'vid': 'V411709', 'txid': 'TX07763', 'signature': '3f7d354d08c966c2a43fb99ba6b9b37d'}
b'\x0eV411709\x0eTX07763@3f7d354d08c966c2a43fb99ba6b9b37d'
TX04619 A85883 A68618 1239
{'vid': 'V411709', 'txid': 'TX04619', 'signature': '284e40858ca6fae089ef74d0

KeyboardInterrupt: 